# Loading dependencies

In [2]:
import numpy
import sklearn
import sklearn.metrics.pairwise
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer

# Data cleaning

The first step of nearly any machine learning analysis project is data cleaning. This is done in order to allow a larger variety of models to work with a predictable input, such that exceptions (in this case special characters such as quotation marks, '[comma]' and others) will not cause any disturbance in the model. The following code loads the data, 'cleans' it, and afterwards sets the entire cleaned data in an array. Comments are added in the code for interpretability. NLTK is also used for cleaning.

In [3]:
## Set an empty list variable

descriptions = []

with open('descriptions.txt', encoding = "utf8") as f:
    for line in f:
        text = line.lower()                                       ## Lowercase all characters
        text = text.replace("[comma]"," ")                        ## Replace [commas] with empty space
        for ch in text:
            if ch < "0" or (ch < "a" and ch > "9") or ch > "z":   ## The cleaning operation happens here, remove all special characters
                text = text.replace(ch," ")
        text = ' '.join(text.split())                             ## Remove double spacing from sentences
        descriptions.append(text)
dataSet = numpy.array(descriptions)

print('After running first results, the following sentence was found : ')
print()
print(dataSet[496])
print()
print()
print('Although this sentence is not meaningful, it will remain in the dataset in order to have consistent results.')

After running first results, the following sentence was found : 

rwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewad rwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdcerfadwerfdewadrwevfderdc

# Model

## Representation - TfIdf Vectorizer

Since the input vector now is 'clean', different representations can be made, which in turn can then be 'trained' to obtain accuracy measures of classification. Used TfIdf vectorizer in order the have the Term frequency, inverse document frequency (which will essentially put less importance on non-informative words suchs as: 'the', 'and', 'a'). We included the stop_words parameter which also strips out unimportant words.

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
TfIdf_dataSet = vectorizer.fit_transform(dataSet)
print("What our Tf-Idf looks like: ","\n")
print(TfIdf_dataSet[0:1])

vectorVocab = vectorizer._validate_vocabulary()

What our Tf-Idf looks like:  

  (0, 3085)	0.18016293756155294
  (0, 1351)	0.10858700066557808
  (0, 3257)	0.3001624823648848
  (0, 2585)	0.34043262183046635
  (0, 2139)	0.11306416211646501
  (0, 4024)	0.21854146699056293
  (0, 1981)	0.25581194154091114
  (0, 3626)	0.31679149916873117
  (0, 1152)	0.24315753998842082
  (0, 4070)	0.2684663430934014
  (0, 1073)	0.30413709761624086
  (0, 2120)	0.2684663430934014
  (0, 1687)	0.06619086894558779
  (0, 1200)	0.26383180239523174
  (0, 1473)	0.31679149916873117
  (0, 1117)	0.16480321090672406
  (0, 4092)	0.16167588241502212


The data is now cleaned and neatly fit into an sparse array. Some basic information about the cleaned array will be provided in the following code.

In [5]:
print('The size of our data set: ', dataSet.size)
print('The dimension of our dataset are: ', dataSet.shape)
print('\n')
print('-- 0th element of our dataSet --', '\n', dataSet[0])
print('\n')
print('-- 1st element of our dataSet --', '\n', dataSet[1])

The size of our data set:  1480
The dimension of our dataset are:  (1480,)


-- 0th element of our dataSet -- 
 round face short and overweight likes to wear jeans and sweaters drinks wine at dinner short liberal overweight short hair eats at whole foods does not work our very much


-- 1st element of our dataSet -- 
 jug ears mustache and beard and long sideburns stylish hair no laugh lines eyes are clear no drugs or alcohol confident a little overweight from double chin


## Distance Measure - Cosine similarity

Now we can safely compute the distance between each document. After sorting, the most similar top 5 documents will be provided. The first vector in the matrix represents the 'base' sentence. The vectors following are the sentences most similar to that 'base' sentence. This should be read per row. For example, the second element of the first row is most similar to the first element of the first row.

In [6]:
## Make use of SKlearn cosine similarity
cosineSimilarity = sklearn.metrics.pairwise.cosine_similarity(TfIdf_dataSet)
print(cosineSimilarity)

[[1.         0.06419899 0.01454109 ... 0.00629138 0.06862054 0.07042177]
 [0.06419899 1.         0.06743771 ... 0.04726818 0.06877843 0.04734656]
 [0.01454109 0.06743771 1.         ... 0.00565678 0.065617   0.02817174]
 ...
 [0.00629138 0.04726818 0.00565678 ... 1.         0.00482428 0.00497692]
 [0.06862054 0.06877843 0.065617   ... 0.00482428 1.         0.07985904]
 [0.07042177 0.04734656 0.02817174 ... 0.00497692 0.07985904 1.        ]]


In [7]:
## Adjust the cosineSimilarity matrix accordingly to sort and get results
numpy.fill_diagonal(cosineSimilarity,1.1)
cosineSimilaritySorted = numpy.argsort((-1*(cosineSimilarity)),axis=1)
top5similar = (cosineSimilaritySorted[:,0:5])
print()
print(top5similar)


[[   0 1454   65   66  406]
 [   1  556  549 1373  944]
 [   2  342    4  288  379]
 ...
 [1477 1372  210  902  681]
 [1478  967  706  669 1084]
 [1479 1341 1144  500  773]]


### Interpretation of the cosine similarity

Following the cosine metric, the first sentence in our dataSet is closest to the 1455 sentence in our data set. Let's see what they both look like:

In [8]:
print('Sentence 1 in the dataSet: ')
print(dataSet[0])
print()
print('Sentence 1455 in the dataSet: ')
print(dataSet[1454])

Sentence 1 in the dataSet: 
round face short and overweight likes to wear jeans and sweaters drinks wine at dinner short liberal overweight short hair eats at whole foods does not work our very much

Sentence 1455 in the dataSet: 
she is older looking and has some wrinkles she has a round face and a round nose has 2 kids has 1 grandkid not very happy likes to drink wine


### Save Results

In [ ]:
numpy.savetxt("results.csv", top5similar.astype(int), fmt='%i', delimiter=",")